# Computational Work 01: Fundamentals of optimizations

###  The quasi-Newton approaches (BFGS, L-BFGS, and DFP) and Levenberg-Marquardt must be applied and compared to the following functions:
* Rosenbrock (2D)
* Rosenbrock (4D or 30D)
* Beale (2D)
* Booth (2D)
* Mathias (2D)
* Ackley (2D)
* Rastrigin (2D)
* Rastrigin (4D or 30D).
#### 1. Table with min, max, mean, and median objective functions values (30 runs with different initial solution for each run) must be presented and discussed.
#### 2.  Best results (solution, number of iterations to converge, number total of objective function evaluations, and objective function value) must be showed.
### Optional:
#### 3. In the 2D case studies, the isolines and convergence line of all optimizers must be presented.

In [133]:
import pandas as pd
from scipy import optimize as opt
import numpy as np
from IPython.display import display

In [134]:
#Does : BFGS
#https://docs.scipy.org/doc/scipy/reference/optimize.minimize-bfgs.html
#results.append(opt.minimize(fun=func,x0=(np.random.uniform(-bound,bound),np.random.uniform(-bound,bound))

#Does L-BFGS
#https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html
#lbfgs = opt.minimize(fun=beale,x0=(np.random.uniform(-10,10),np.random.uniform(-10,10)),bounds=((-10,10),(-10,10)) ,method='L-BFGS-B')

#DFP ?
#https://docs.scipy.org/doc/scipy/reference/optimize.minimize-powell.html ??

#Does Levenberg-Maquardt
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.least_squares.html

#pd.set_option("display.precision",15)
pd.set_option("display.precision", 3)
pd.set_option('display.float_format', '{:.2g}'.format)

def OptmizeResult_to_Series(results: list, nome: str):
    ''' Pega o resultado de opt.minimize. Retorna:
        \n data: pd.Series com a min, máx, mean e median
        \nbest: pd.Series com o melhor valor obtido
     '''

    try:
        method_data=pd.DataFrame({
        "value": [ result.fun for result in results],
        "iterations": [ result.nit for result in results],
        "func_eval": [ result.nfev for result in results],
        "solutions": [ result.x for result in results]})
    except:
        method_data=pd.DataFrame({
        "value": [ result.fun for result in results],
        "iterations": '',
        "func_eval": [ result.nfev for result in results],
        "solutions": [ result.x for result in results]})

    method_data.sort_values(by="value",inplace=True)
    data=pd.Series({
        "nome":nome,
        "min":method_data["value"].min(),
        "max":method_data["value"].max(),
        "mean":method_data["value"].mean(),
        "median": method_data["value"].median(),
     })
    
    best=pd.Series(method_data.iloc[0])
    best["nome"]=nome
    best =best[["nome","value","iterations","func_eval","solutions"]]

    return data, best


def apply_methhods(func, search_space: float):
    '''Aplica os metodos nas funções objetiva. Metodos implementados: BFGS, LBFGS, Powell, Minimos Quadrados (Levenberg-Maquart)
        \n Retorna:
        \n *data*: pd.DataFrame com a min, máx, mean e median
        \n *best*: pd.DataFrame com o melhor valor obtido
        
    '''

    bfgs =[]
    lbfgs=[]
    powell =[]
    lq=[]
    for _ in np.arange(30):
            x00=np.random.uniform(-search_space,search_space)
            x01=np.random.uniform(-search_space,search_space)
            
            bfgs.append(opt.minimize(fun=func,x0=(x00,x01), method='BFGS'))
            lbfgs.append(opt.minimize(fun=func,x0=(x00,x01), bounds=((-search_space,search_space),(-search_space,search_space)) ,method='L-BFGS-B'))
            powell.append(opt.minimize(func, x0=(x00,x01) ,method="Powell",bounds=((-search_space,search_space),(-search_space,search_space))))
            lq.append(opt.least_squares(func,x0=(x00,x01),bounds=((-search_space,search_space))))

            
    bfgs_data,bfgs_best= OptmizeResult_to_Series(bfgs, 'bfgs')
    lbfgs_data, lbfgs_best= OptmizeResult_to_Series(lbfgs,'lbfgs')
    powell_data, powell_best = OptmizeResult_to_Series(powell,"powell")
    lq_data,lq_best= OptmizeResult_to_Series(lq,"Minimos Quadrados")

    data = pd.DataFrame([bfgs_data,lbfgs_data,powell_data,lq_data])
    best= pd.DataFrame([bfgs_best,lbfgs_best,powell_best,lq_best])
    return data, best





### Rosenbrock 2D

In [135]:
rosenbrock = lambda x: 100 * (x[1] - x[0]**2)**2
search_space = 10

data, best = apply_methhods(rosenbrock,search_space=search_space)
display(data)
display(best)


,nome,min,max,mean,median
0,bfgs,4.1e-18,5.4e-12,6.9e-13,1.6e-13
1,lbfgs,1.1e-16,1.4e-13,3e-14,2.1e-14
2,powell,0,5.5e-06,3.5e-07,1.4e-27
3,Minimos Quadrados,[2.587383406549521e-09],[8.24792622536817e-08],[3.27787916283656e-08],3.1e-08


,nome,value,iterations,func_eval,solutions
10,bfgs,4.1e-18,6,173,"[2.663637441379083, 7.094964418913271]"
4,lbfgs,1.1e-16,8,39,"[-1.4589734760538946, 2.128603604892285]"
19,powell,0,2,44,"[2.0648003894157303, 4.263400648131351]"
17,Minimos Quadrados,[2.587383406549521e-09],,19,"[-0.3461945792177936, 0.11984560004690424]"


### Rosenbrock 4D

In [136]:
#rosenbrock = lambda x: np.array([10 * (x[1] - x[0]**2), (1 - x[0])])

### Beale

In [137]:
beale= lambda x: ( np.square(1.5-x[0] +x[0]*x[1] ) +
        np.square(2.25 -x[0] + x[0]*np.square( x[1]) )+
        np.square(2.625 -  x[0] +x[0]*np.power(x[1],3) ))

search_space = 4.5

data, best = apply_methhods(beale,search_space=search_space)
display(data)
display(best)

,nome,min,max,mean,median
0,bfgs,1.6e-14,7.4,0.43,1.2e-11
1,lbfgs,3.1e-15,0.77,0.15,1.2e-13
2,powell,1.2e-15,9.9,4.1,1.4
3,Minimos Quadrados,[0.000508653278022604],[9.731996485657897],[0.8916549788750127],0.0086


,nome,value,iterations,func_eval,solutions
7,bfgs,1.6e-14,14,54,"[2.9999999856724036, 0.500000023079733]"
1,lbfgs,3.1e-15,19,75,"[2.9999998989894117, 0.4999999830205596]"
9,powell,1.2e-15,9,332,"[2.9999999438827256, 0.49999999163847525]"
26,Minimos Quadrados,[0.000508653278022604],,200,"[3.0578316340960265, 0.5145025787258458]"


## Booth

In [138]:
booth = lambda x: np.square(x[0]+2*x[1]-7) + np.square(2*x[0]+x[1]-5)

search_space = 10

data, best = apply_methhods(booth,search_space=search_space)
display(data)
display(best)



,nome,min,max,mean,median
0,bfgs,1.6e-17,2e-11,1.8e-12,3.1e-14
1,lbfgs,1.4e-16,8.2e-12,1.2e-12,9.6e-14
2,powell,0,0.42,0.045,7.9e-31
3,Minimos Quadrados,[5.2523952371059934e-08],[5.827110111452291e-07],[3.590925090749762e-07],3.7e-07


,nome,value,iterations,func_eval,solutions
19,bfgs,1.6e-17,6,21,"[1.0000000015774937, 3.0000000002702425]"
28,lbfgs,1.4e-16,5,18,"[0.9999999972229396, 2.9999999972232287]"
22,powell,0,3,45,"[1.0000000000000002, 3.0]"
26,Minimos Quadrados,[5.2523952371059934e-08],,38,"[1.0001703252764091, 2.999871546093754]"


### Matyas

In [145]:
matyas= lambda x: 0.26*(np.square(x[0]+np.square(x[1])))-0.48*x[0]*x[1]
search_space = 10

data, best = apply_methhods(matyas,search_space=search_space)
display(data)
display(best)

,nome,min,max,mean,median
0,bfgs,-8e+16,-0.007,-1.6e+16,-5.4e+11
1,lbfgs,-16,-0.007,-13,-16
2,powell,-16,0.76,-14,-16
3,Minimos Quadrados,[-3.150478367075493e-08],[5.381862755449163e-08],[1.2389794204295149e-09],5.7e-11


,nome,value,iterations,func_eval,solutions
28,bfgs,-8e+16,400,1581,"[-304580969794.655, -551935.0428572403]"
29,lbfgs,-16,13,57,"[-10.0, -3.371769789615052]"
12,powell,-16,3,223,"[-9.999999999998977, -3.3717766687280712]"
24,Minimos Quadrados,[-3.150478367075493e-08],,32,"[-1.850128852977513, -2.13289281800563]"


### Rastrigin

In [140]:
rastrigin= None


### Ackley 

In [147]:
ackley = lambda x: (-20*np.exp(-0.2*np.sqrt(0.5*(np.square(x[0]+np.square(x[1]))))) 
                    -np.exp(np.cos(2*np.pi*x[0]) +np.cos(2*np.pi*x[0]*x[1]))+ np.e +20
                    )
search_space = 32

data, best = apply_methhods(ackley,search_space=search_space)
display(data)
display(best)


,nome,min,max,mean,median
0,bfgs,-2.8,15,12,15
1,lbfgs,-3.3,15,11,15
2,powell,-4.7,20,3.8,-2.5
3,Minimos Quadrados,[-7.325978437222602e-07],[21.562905115135493],[12.910035774969531],15


,nome,value,iterations,func_eval,solutions
28,bfgs,-2.8,10,164,"[-5.029526914465416, -2.3842121350891543]"
2,lbfgs,-3.3,9,57,"[-19.028303478730905, 4.4142513266461]"
15,powell,-4.7,2,41,"[-3.6859404417555197e-13, -4.440892098500626e-15]"
28,Minimos Quadrados,[-7.325978437222602e-07],,29,"[-4.026414402182045, -2.0428378898840336]"
